In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS

In [6]:
loader = PyPDFLoader("NIPS-2017-attention-is-all-you-need-Paper.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    separators=['\n', '\n\n', ','],
    chunk_overlap=200
)
documents = text_splitter.split_documents(documents)
documents[:5]
print(len(documents))

43


In [9]:
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\PMLS\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PMLS\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import numpy as np
np.array(huggingface_embeddings.aembed_query(documents[0].page_content))

array(<coroutine object Embeddings.aembed_query at 0x000001FB77314200>,
      dtype=object)

In [13]:
import numpy as np
print(np.array(huggingface_embeddings.embed_query(
    documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(
    documents[0].page_content)).shape)

[-7.17908219e-02 -1.25976473e-01  4.42448491e-03 -3.68517302e-02
  2.36916039e-02  3.58989649e-02 -6.33518249e-02  1.89670641e-02
  1.12373739e-01 -6.06929436e-02  2.29640715e-02 -5.51820844e-02
 -7.19434470e-02  1.01218391e-02 -5.58682494e-02 -1.63020229e-03
  3.86950187e-02  7.95766637e-02 -6.54759631e-02 -9.69664156e-02
  1.78688113e-02  2.30634790e-02  1.42895766e-02  2.12099124e-02
  2.34116940e-03  3.48839425e-02 -7.75443912e-02 -5.70324510e-02
 -1.10436417e-02  2.94086686e-03  3.61996740e-02 -4.30017933e-02
 -2.80836839e-02  8.00447762e-02 -5.03871515e-02  1.10283516e-01
 -9.54435468e-02 -2.12040301e-02 -1.46536026e-02 -7.19997659e-02
  1.25844739e-02  1.58007368e-02 -3.65340486e-02  1.63824111e-02
  1.21759474e-01  3.91672589e-02  6.51316866e-02 -1.60369705e-02
  2.03093532e-02 -3.36510539e-02 -7.29919970e-02  4.37113345e-02
 -3.06430031e-02  8.78230333e-02 -1.36171635e-02 -5.19783329e-03
 -4.31595147e-02 -1.89790931e-02 -3.69802527e-02 -6.66522607e-02
 -7.62080252e-02 -3.43455

In [24]:
vectorstore = FAISS.from_documents(
    documents, huggingface_embeddings)

In [25]:
# Query using Similarity Search
query = "Encoder and Decoder Stacks?"
relevant_docments = vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The ﬁrst is a multi-head self-attention mechanism, and the second is a simple, position-
2


In [31]:
# Query using Similarity Search
query2 = " Embeddings and Softmax"
relevant_docments2 = vectorstore.similarity_search(query2)

print(relevant_docments2[0].page_content)

of the softmax which correspond to illegal connections. See Figure 2.
3.3 Position-wise Feed-Forward Networks
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully
connected feed-forward network, which is applied to each position separately and identically. This
consists of two linear transformations with a ReLU activation in between.
FFN(x) = max(0,xW 1+b1)W2+b2 (2)
While the linear transformations are the same across different positions, they use different parameters
from layer to layer. Another way of describing this is as two convolutions with kernel size 1.
The dimensionality of input and output is dmodel = 512 , and the inner-layer has dimensionality
dff= 2048 .
3.4 Embeddings and Softmax
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transfor-
